<a href="https://colab.research.google.com/github/brahma1995/image_embeddings_matching/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [ ]:
pic_one = '/content/porcelain-dinner-plate-in-black-by-the-artment-porcelain-dinner-plate-in-black-by-the-artment-7mkfoj.jpg'
pic_two = '/content/dinerhalf.jpg'

In [ ]:
import torch
from tqdm import tqdm
from torchvision import models
from torchvision import transforms

# for this prototype we use no gpu, cuda= False and as model resnet18 to obtain feature vectors

class Img2VecResnet18():
    def __init__(self):

        self.device = torch.device("cpu")
        self.numberFeatures = 512
        self.modelName = "resnet-18"
        self.model, self.featureLayer = self.getFeatureLayer()
        self.model = self.model.to(self.device)
        self.model.eval()
        self.toTensor = transforms.ToTensor()

        # normalize the resized images as expected by resnet18
        # [0.485, 0.456, 0.406] --> normalized mean value of ImageNet, [0.229, 0.224, 0.225] std of ImageNet
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def getVec(self, img):
        image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
        embedding = torch.zeros(1, self.numberFeatures, 1, 1)

        def copyData(m, i, o): embedding.copy_(o.data)

        h = self.featureLayer.register_forward_hook(copyData)
        self.model(image)
        h.remove()

        return embedding.numpy()[0, :, 0, 0]

    def getFeatureLayer(self):

        cnnModel = models.resnet18(pretrained=True)
        layer = cnnModel._modules.get('avgpool')
        self.layer_output_size = 512

        return cnnModel, layer




In [ ]:
# generate vectors for all the images in the set
img2vec = Img2VecResnet18()
Img1 = Image.open(pic_one)
pic_one_vector  = img2vec.getVec(Img1)
Img2 = Image.open(pic_two)
pic_two_vector = img2vec.getVec(Img2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 49.7MB/s]


In [ ]:
import torch
import torch.nn as nn

# Assuming pic_one_vector and pic_two_vector are NumPy arrays
pic_one_vector_tensor = torch.tensor(pic_one_vector, dtype=torch.float)
pic_two_vector_tensor = torch.tensor(pic_two_vector, dtype=torch.float)

# Create the nn.CosineSimilarity object
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

# Calculate the cosine similarity between pic_one_vector and pic_two_vector
cos_sim = cos(pic_one_vector_tensor.unsqueeze(0), pic_two_vector_tensor.unsqueeze(0))

# Print the cosine similarity
print('\nCosine similarity: {0}\n'.format(cos_sim.item()))



Cosine similarity: 0.9903527498245239

